**SIMULAÇÃO DINÂMICA DE UMA BICICLETA ELÉTRICA AUTÔNOMA**

**Este projeto simula uma bicicleta elétrica autônoma em circuito em linha reta e em um circuito fechado com subidas, descidas e curvas suaves. A simulação foi desenvolvida do zero com foco em aplicações reais de dinâmica veicular e controle autônomo. O objetivo principal é demonstrar competência técnica em simulação física, controle autônomo e análise de dados veiculares, em um contexto aplicável à indústria automotiva elétrica e de mobilidade inteligente. Foi utilizado:**

**Modelo físico longitudinal:** com forças resistivas (peso, inclinação, arrasto, eficiência).

**Modelo cinemático de bicicleta:** para atualização de posição e orientação.

**Controladores PID (velocidade):** e **Stanley (trajetória)**, calibrados para estabilidade.

**Simulação energética:** com torque, potência e consumo da bateria a cada instante.

**Geração e análise de logs:** com variáveis físicas, agrupadas por setor da pista.

**Insights visuais:** sobre desempenho energético, esforço do motor e rendimento por setor.

In [ ]:
# Importando Bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import warnings
plt.rcParams['figure.figsize'] = (10, 5)
warnings.filterwarnings('ignore')

**O modelo longitudinal descreve o movimento da bicicleta na direção de avanço (reta), levando em conta as forças que agem ao longo da linha do solo.**

**Vamos começar desenvolvendo um modelo físico completo para simular a dinâmica longitudinal de uma bicicleta elétrica, considerando os principais elementos que influenciam seu movimento em linha reta. O modelo foi implementado em Python com base na segunda lei de Newton, onde a aceleração resulta da soma das forças atuantes. As seguintes componentes físicas foram modeladas:**

**Força motriz gerada pelo motor elétrico, com curva de torque simplificada em função da rotação (RPM).**

**Força de arrasto aerodinâmico, proporcional ao quadrado da velocidade.**

**Resistência ao rolamento, dependente da massa, do coeficiente de rolamento e da inclinação da pista.**

**Componente gravitacional da inclinação, atuando como força adicional em subidas e descidas.**

**Frenagem, incluindo a possibilidade de regeneração de energia com eficiência ajustável.**

**A energia da bateria é atualizada a cada passo de simulação, levando em conta o consumo por tração e a recuperação durante frenagens. O modelo fornece, a cada instante, os valores atualizados de posição, velocidade, aceleração e nível da bateria. Este modelo vai servir como base para os módulos de controle, visualização e otimização, permitindo simular o comportamento físico de veículos leves elétricos sob diferentes condições operacionais.**

In [ ]:
class EVLongitudinalModel: # Essa função __init__ inicializa os parâmetros físicos e o estado interno do modelo longitudinal da bicicleta elétrica
    def __init__(self, mass_kg, wheel_radius_m, CdA, Crr, battery_capacity_Wh, motor_efficiency=0.9, regen_efficiency=0.6):
        # Parâmetros fixos do sistema
        self.mass = mass_kg # massa total da bicicleta + piloto
        self.r = wheel_radius_m # raio da roda traseira em metros
        self.CdA = CdA # Produto do coeficiente aerodinâmico (Cd) e da área frontal (A) (arrasto)
        self.Crr = Crr # Coeficiente de resistência ao rolamento
        self.g = 9.81 # Gravidade
        self.rho = 1.225 # Densidade do ar nível do mar
        self.battery_capacity_Wh = battery_capacity_Wh # Capacidade total da bateria (em Wh)
        self.motor_eff = motor_efficiency # Eficiência do motor elétrico (0.9 = 90%) 90% da energia elétrica vira força útil.
        self.regen_eff = regen_efficiency # Eficiência da frenagem regenerativa (0.6 = 60%) 60% da energia da frenagem vira recarga.

        # Estado inicial
        self.v = 0.0  # m/s velocidade inicial da bicicleta
        self.x = 0.0  # posição inicial (m)
        self.battery_Wh = battery_capacity_Wh # Energia atual disponível na bateria. Começa cheia (igual à capacidade total).

    # Simular de forma simplificada como o torque do motor varia conforme a rotação (RPM). Essa curva é típica de motores elétricos: torque alto no começo, depois vai caindo.
    def torque_curve(self, rpm): # Comportamento do motor elétrico em função da rotação (recebe a rotação como entrada)
        # Curva simplificada de torque em função da rotação
        if rpm < 100:
            return 40  # torque máximo até 100 RPM
        elif rpm < 300:
            return 40 - 0.1 * (rpm - 100)  # torque decai linearmente
        else:
            return 20  # torque mínimo

    def step(self, throttle, brake_force, incline_deg=0.0, dt=0.1): # Função principal de simulação dinâmica.
        # throttle - nível de aceleração (0 a 1)
        # brake_force - força de frenagem (em Newtons)
        # incline_deg - ângulo da inclinação da pista em graus
        # dt - passo de tempo (duração da simulação, em segundos)

        # Convertendo velocidade longitudinal (self.v) para RPM
        rpm = (self.v / (2 * np.pi * self.r)) * 60
        torque = self.torque_curve(rpm)

        # Calcula a força motriz total gerada pelo motor e aplica à bicicleta.
        F_mot = (torque / self.r) * throttle * self.motor_eff

        # Forças resistivas
        theta = np.deg2rad(incline_deg) # Converte o ângulo de inclinação da pista (em graus) para radianos, pois funções trigonométricas do NumPy esperam radianos.
        F_drag = 0.5 * self.rho * self.CdA * self.v**2 # Arrasto aerodinâmico
        F_rr = self.Crr * self.mass * self.g * np.cos(theta) # Resistência ao rolamento (depende do pneu e da da pista)
        F_grade = self.mass * self.g * np.sin(theta) # Força da inclinação (subida/descida) puxa para trás se for subida, ou empurra para frente se for descida.
        F_brake = brake_force # Força de frenagem total


        # Aqui a física vira movimento de verdade, é a hora que o modelo responde às forças calculadas e atualiza a velocidade e a posição.
        # Dinâmica: soma das forças
        F_total = F_mot - F_drag - F_rr - F_grade - F_brake
        a = F_total / self.mass # Calculo da aceleração

        # Atualiza velocidade e posição
        self.v += a * dt # Atualiza a velocidade linear da bicicleta com base na aceleração.
        self.v = max(self.v, 0)  # evita valor negativo
        self.x += self.v * dt # Atualiza a posição longitudinal (em metros)

        # Atualiza bateria
        power_W = F_mot * self.v # Potência mecânica instantânea entregue pelo motor (em watts)
        energy_Wh = (power_W * dt) / 3600 # Energia consumida nesse passo (em Wh). O divisor 3600 converte de joules para watt-hora (Wh).
        if throttle > 0: # Se há aceleração (throttle > 0), o sistema gasta energia da bateria.
            self.battery_Wh -= energy_Wh # Subtrai a energia consumida do estado atual da bateria
        elif a < 0 and self.v > 0:  # Se estiver desacelerando (a < 0) e ainda com velocidade positiva, considera que há recuperação de energia.
            self.battery_Wh += self.regen_eff * energy_Wh # Só uma fração da energia é recuperada, controlada por regen_eff (ex: 0.6 = 60%).

        # Evita ultrapassar capacidade. Garante que o nível da bateria nunca passe de 100% nem vá abaixo de 0
        self.battery_Wh = np.clip(self.battery_Wh, 0, self.battery_capacity_Wh)

        return {
            'position_m': self.x,
            'speed_mps': self.v,
            'accel_mps2': a,
            'battery_Wh': self.battery_Wh
        }


In [ ]:
# Criando o modelo com parâmetros realistas
modelo = EVLongitudinalModel(
    mass_kg=100,                  # bicicleta + piloto
    wheel_radius_m=0.3,           # raio típico de roda
    CdA=0.5,                      # coeficiente aerodinâmico × área frontal
    Crr=0.005,                    # coeficiente de resistência ao rolamento
    battery_capacity_Wh=500,      # capacidade da bateria
    motor_efficiency=0.9,
    regen_efficiency=0.6
)

# Parâmetros da simulação
dt = 0.1         # passo de tempo (s)
tempo_total = 30 # segundos
passos = int(tempo_total / dt)

# Inicializar listas para armazenar resultados
tempos = []
velocidades = []
aceleracoes = []
posicoes = []
bateria = []

# Loop de simulação
for i in range(passos):
    t = i * dt
    resultado = modelo.step(throttle=0.8, brake_force=0, incline_deg=0.0, dt=dt)

    tempos.append(t)
    velocidades.append(resultado['speed_mps'])
    aceleracoes.append(resultado['accel_mps2'])
    posicoes.append(resultado['position_m'])
    bateria.append(resultado['battery_Wh'])

In [ ]:
# Velocidade ao longo do tempo
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(tempos, velocidades, label='Velocidade (m/s)', color='royalblue')
plt.ylabel('Velocidade (m/s)')
plt.grid(True)
plt.legend()

**O gráfico acima mostra a evolução da velocidade longitudinal da bicicleta elétrica ao longo de 30 segundos, com aceleração constante (throttle = 0.8) e sem inclinação ou frenagem. Observamos um crescimento inicial rápido da velocidade, seguido por uma desaceleração gradual na taxa de crescimento até atingir um patamar próximo da estabilidade. Esse comportamento indica o ponto de equilíbrio dinâmico, onde a força motriz do motor é compensada pelas forças resistivas (arrasto aerodinâmico, resistência ao rolamento e força gravitacional em terreno plano). Esse resultado está de acordo com a física esperada de sistemas de propulsão elétrica sob aceleração constante e demonstra o correto funcionamento do modelo longitudinal implementado.**

In [ ]:
# Aceleração ao longo do tempo
plt.figure(figsize=(14, 10))
plt.subplot(3, 1, 2)
plt.plot(tempos, aceleracoes, label='Aceleração (m/s²)', color='green')
plt.ylabel('Aceleração (m/s²)')
plt.grid(True)
plt.legend()

**O gráfico acima apresenta a evolução da aceleração longitudinal da bicicleta elétrica durante 30 segundos de simulação, com aceleração constante definida por throttle = 0.8 (80% da potência do motor) e sem aplicação de frenagem. Inicialmente, a aceleração é elevada, pois a força motriz do motor supera amplamente as resistências (aerodinâmica, rolamento e gravidade). No entanto, à medida que a velocidade aumenta, o arrasto aerodinâmico cresce com o quadrado da velocidade, exigindo mais esforço do motor para manter o mesmo ritmo de aceleração. Esse comportamento provoca uma redução progressiva da aceleração, que tende a zero conforme o sistema se aproxima do equilíbrio dinâmico, onde as forças resistivas igualam a força de tração. A curva descendente suave confirma o correto funcionamento do modelo físico e representa com fidelidade o comportamento real de veículos elétricos sob aceleração contínua.**

In [ ]:
# Consumo da bateria ao longo do tempo
plt.figure(figsize=(14, 10))
plt.subplot(3, 1, 3)
plt.plot(tempos, bateria, label='Bateria (Wh)', color='orange')
plt.xlabel('Tempo (s)')
plt.ylabel('Energia Restante (Wh)')
plt.grid(True)
plt.legend()

**Acima observamos a evolução da energia restante da bateria (Wh) durante os 30 segundos iniciais de simulação com aceleração constante (throttle = 0.8) e sem frenagem regenerativa. Observa-se uma redução progressiva e suave da energia ao longo do tempo, como esperado em um cenário de aceleração contínua. O consumo não é linear, pois está diretamente relacionado à potência do motor, que depende da força motriz e da velocidade instantânea. Como a velocidade aumenta no início e depois se estabiliza, o consumo energético também tende a se suavizar. A inclinação negativa da curva confirma que o modelo está contabilizando corretamente o consumo de energia mecânica convertido em energia elétrica, respeitando a eficiência do sistema motriz, esse comportamento é típico de veículos elétricos reais e reforça a fidelidade física do modelo.**

**Agora vamos implementar o modelo cinemático da bicicleta, que vai complementar o modelo longitudinal e nos permitir simular a trajetória espacial (X, Y) com controle de direção.**

**O modelo cinemático de bicicleta (também chamado “modelo de carro simples”) assume que:**

**- a bicicleta se move rigidamente com um único ponto de virada (pivot).**
**- o controle é feito via ângulo de esterço.**
**- a entrada principal é a velocidade longitudinal.**

**Trata-se de uma abstração baseada na geometria do veículo, que descreve seu movimento no plano 2D a partir da velocidade longitudinal e do ângulo de esterço (direção do guidão). A modelagem é fundamentada na cinemática de Ackermann.**


In [ ]:
class KinematicBicycleModel: # Essa classe serve para simular o movimento espacial (X, Y) do veículo com base na velocidade e no ângulo de esterço.
    def __init__(self, wheelbase_m):
        self.L = wheelbase_m  # entre-eixos (distância entre rodas)

        # Estado inicial
        self.x = 0.0      # posição X horizontal
        self.y = 0.0      # posição Y vertical
        self.theta = 0.0  # orientação (ângulo de guinada, em rad) = 0 para a direita (eixo x positivo)

    def step(self, v, delta, dt=0.1):
        # v: velocidade longitudinal (m/s)
        # delta: ângulo de esterço (rad)

        # Calcula o deslocamento em X e Y com base na orientação atual (theta) e velocidade.
        dx = v * np.cos(self.theta) * dt
        dy = v * np.sin(self.theta) * dt
        dtheta = (v / self.L) * np.tan(delta) * dt # Atualiza o ângulo de orientação da bicicleta no plano.
        # Atualiza a nova posição da bicicleta em x e y
        self.x += dx
        self.y += dy
        self.theta += dtheta

        return {
            'x': self.x,
            'y': self.y,
            'theta': self.theta
        }


In [ ]:
# Instancia o modelo cinemático com entre-eixos típico
bike_model = KinematicBicycleModel(wheelbase_m=1.0)

# Parâmetros da simulação
v_const = 5.0  # m/s
delta_deg = 5  # ângulo de esterço em graus
delta_rad = np.deg2rad(delta_deg)
dt = 0.1
tempo_total = 30
passos = int(tempo_total / dt)

# Armazenar resultados
traj_x = []
traj_y = []
angulos = []
tempos = []

# Loop de simulação
for i in range(passos):
    t = i * dt
    resultado = bike_model.step(v=v_const, delta=delta_rad, dt=dt)

    traj_x.append(resultado['x'])
    traj_y.append(resultado['y'])
    angulos.append(np.rad2deg(resultado['theta']))
    tempos.append(t)

In [ ]:
# Plot da trajetória no plano XY
plt.figure(figsize=(8, 6))
plt.plot(traj_x, traj_y, color='darkred', linewidth=2)
plt.xlabel('Posição X (m)')
plt.ylabel('Posição Y (m)')
plt.title('Trajetória da Bicicleta - Modelo Cinemático (Curva com 5°)')
plt.axis('equal')
plt.grid(True)
plt.show()

**O gráfico acima representa a trajetória bidimensional (X, Y) de uma bicicleta simulada utilizando o modelo cinemático baseado em Ackermann, com velocidade constante e ângulo de esterço fixo de 5°. Como esperado, o veículo descreve um movimento circular uniforme, resultado da combinação entre a geometria do entre-eixos e o ângulo de direção aplicado. Esse comportamento é típico de veículos sob esterçamento constante, confirmando o funcionamento correto do modelo. Esse modelo serve como base para controle de trajetória, navegação autônoma e análise de estabilidade direcional em veículos leves.**

**Para a próxima etapa, vamos fazer o acoplamento robusto entre os modelos longitudinal e cinemático. Vamos conectar para que a saída do modelo longitudinal (velocidade) vire a entrada do modelo cinemático; O movimento no plano XY dependa das condições reais do percurso: inclinação, curvas, frenagem, potência; A simulação evolua como um sistema real: um veículo que se move com massa, motor, forças físicas e direção.**

**Entradas:**

**throttle (aceleração do motor)**

**brake_force (frenagem)**

**incline_deg (inclinação da pista)**

**steering_angle_deg (ângulo de direção do guidão)**

**O modelo longitudinal simula a física real (forças, aceleração, consumo) e retorna: Velocidade (v).**

**O modelo cinemático recebe essa v como entrada, junto com o steering_angle, e atualiza: Posição (x, y); Orientação (theta).**

**Com isso será construído um loop integrado onde o motor acelera de forma realista e a posição evolui no plano XY. Poderemos plugar curvas reais, variação de inclinação ou mudança de direção a cada instante, tudo coordenado.**

In [ ]:
# Instanciar modelos
modelo_dyn = EVLongitudinalModel(
    mass_kg=100, wheel_radius_m=0.3,
    CdA=0.5, Crr=0.005, battery_capacity_Wh=500
)
modelo_kin = KinematicBicycleModel(wheelbase_m=1.0)

# Parâmetros da simulação
dt = 0.1
tempo_total = 30
passos = int(tempo_total / dt)

# Inputs fixos para este teste
throttle = 0.8
brake = 0.0
incline = 0.0
steering_angle_deg = 5
delta_rad = np.deg2rad(steering_angle_deg)

# Armazenamento
traj_x, traj_y, velocidades = [], [], []

# Loop de simulação acoplada
for _ in range(passos):
    # Simula a física longitudinal
    result_dyn = modelo_dyn.step(throttle=throttle, brake_force=brake, incline_deg=incline, dt=dt)
    v = result_dyn['speed_mps']  # saída usada como entrada cinemática

    # Simula a movimentação no espaço
    result_kin = modelo_kin.step(v=v, delta=delta_rad, dt=dt)

    # Armazena resultados
    traj_x.append(result_kin['x'])
    traj_y.append(result_kin['y'])
    velocidades.append(v)

In [ ]:
# Simulação já feita previamente (traj_x, traj_y, velocidades)
velocidades_array = np.array(velocidades)
norm = plt.Normalize(velocidades_array.min(), velocidades_array.max())
cmap = plt.cm.plasma
cores = cmap(norm(velocidades_array))

# Criar figura e eixo
fig, ax = plt.subplots(figsize=(10, 6))

# Desenhar a trajetória com gradiente de velocidade
for i in range(1, len(traj_x)):
    ax.plot(traj_x[i-1:i+1], traj_y[i-1:i+1], color=cores[i], linewidth=2)

# Criar mapeamento para barra de cor
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Necessário para evitar warning

# Adicionar barra de cores no eixo correto
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Velocidade (m/s)")

# Estética do gráfico
ax.set_xlabel("Posição X (m)")
ax.set_ylabel("Posição Y (m)")
ax.set_title("Trajetória no Plano XY com Velocidade Variável")
ax.set_aspect('equal')
ax.grid(True)

plt.show()

**O gráfico acima mostra a trajetória da bicicleta simulada no plano (X, Y), utilizando o modelo acoplado entre a dinâmica longitudinal e a cinemática veicular. A velocidade é calculada a cada instante a partir das forças físicas (motor, arrasto, resistência ao rolamento e inclinação), e usada como entrada para o modelo cinemático, que atualiza a posição e orientação da bicicleta. O ângulo de esterço foi mantido constante em 5°, o que leva o veículo a descrever um arco de raio fixo, resultando, após a fase inicial, em uma trajetória aproximadamente circular. A coloração do traçado representa a velocidade ao longo da trajetória, variando de tons escuros (velocidade baixa, início da aceleração) até tons claros (velocidade estabilizada). Isso evidencia o comportamento natural do sistema, onde a bicicleta acelera inicialmente e depois atinge um regime próximo ao equilíbrio dinâmico.**

In [ ]:
# Converte listas para numpy arrays se necessário
traj_x = np.array(traj_x)
traj_y = np.array(traj_y)
velocidades = np.array(velocidades)

# Velocidade máxima atingida
v_max = velocidades.max()

# Raio de curva estimado (média da distância do centro do círculo aos pontos da trajetória)
# Estimar centro (simples: média dos pontos, funciona bem para círculos regulares)
xc = np.mean(traj_x)
yc = np.mean(traj_y)
raios = np.sqrt((traj_x - xc)**2 + (traj_y - yc)**2)
raio_medio = np.mean(raios)

# Distância total percorrida (soma dos trechos entre pontos consecutivos)
dx = np.diff(traj_x)
dy = np.diff(traj_y)
distancias = np.sqrt(dx**2 + dy**2)
distancia_total = np.sum(distancias)

# Criando a tabela
tabela = pd.DataFrame({
    'Métrica': ['Velocidade Máxima (m/s)', 'Raio de Curva Estimado (m)', 'Distância Total Percorrida (m)'],
    'Valor': [v_max, raio_medio, distancia_total]
})

# Exibindo a tabela
plt.figure(figsize=(10, 3))
sns.heatmap(tabela.set_index('Métrica').T, annot=True, fmt=".2f", cmap="YlGnBu", cbar=False)
plt.title("Resumo da Simulação Cinemática")
plt.yticks(rotation=0)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


**A tabela acima resume os principais indicadores extraídos da simulação do modelo acoplado (longitudinal + cinemático):**

**Velocidade Máxima:** O modelo atingiu aproximadamente 10.81 m/s, compatível com a aceleração gradual fornecida pelo motor e a resistência do sistema.

**Raio de Curva Estimado:** Com um ângulo de esterço fixo de 5° e entre-eixos de 1.0 m, o raio médio da trajetória foi de 11.21 m, confirmando o padrão circular do percurso.

**Distância Total Percorrida:** A trajetória resultou em um deslocamento acumulado de 233.01 m, coerente com os 30 segundos de simulação sob aceleração constante.

**Esses dados permitem validar o comportamento do sistema, avaliar a eficiência do modelo longitudinal e a fidelidade da trajetória gerada pelo modelo cinemático.**

In [ ]:
# Recalcula os pontos da curva
points = np.array([traj_x, traj_y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# Normalização e colormap
norm = Normalize(vmin=velocidades.min(), vmax=velocidades.max())
cmap = plt.cm.plasma

# Criação da linha colorida
lc = LineCollection(segments, cmap=cmap, norm=norm)
lc.set_array(velocidades)
lc.set_linewidth(2)

# Gráfico
fig, ax = plt.subplots(figsize=(7, 6))
line = ax.add_collection(lc)
cbar = fig.colorbar(line, ax=ax)
cbar.set_label("Velocidade (m/s)")

ax.set_xlim(traj_x.min()-1, traj_x.max()+1)
ax.set_ylim(traj_y.min()-1, traj_y.max()+1)
ax.set_title("Trajetória no Plano XY - Perfil Agressivo (Acelera e Freia)")
ax.set_xlabel("Posição X (m)")
ax.set_ylabel("Posição Y (m)")
ax.set_aspect('equal')
ax.grid(True)
plt.tight_layout()
plt.show()


**No gráfico acima foi simulada a trajetória da bicicleta no plano XY com um perfil de velocidade agressivo, onde o veículo inicia com aceleração, atinge um pico e posteriormente passa por uma fase de frenagem. A curva descrita representa uma trajetória circular modificada pelas variações de velocidade.**

**Regiões roxas:** baixa velocidade inicial;

**Regiões amarelas:** velocidade mais alta, representando o pico antes da frenagem;

**Transições:** desaceleração progressiva durante a trajetória.

**Essa simulação aproxima ainda mais o comportamento do modelo ao de veículos reais em situações urbanas ou esportivas, onde aceleração e frenagem coexistem em curvas.**

In [ ]:
# Velocidade máxima
v_max = np.max(velocidades)

# Estimativa do raio de curva médio
# Usa distância entre os pontos e variação de ângulo
dx = np.diff(traj_x)
dy = np.diff(traj_y)
ds = np.sqrt(dx**2 + dy**2)
distancia_total = np.sum(ds)

# Estimativa do raio médio: circunferência = 2πr => r ≈ distância_total / (2π)
raio_estimado = distancia_total / (2 * np.pi)

# Criar a tabela
tabela = pd.DataFrame({
    'Métrica': ['Velocidade Máxima (m/s)', 'Raio de Curva Estimado (m)', 'Distância Total Percorrida (m)'],
    'Valor': [round(v_max, 2), round(raio_estimado, 2), round(distancia_total, 2)]
})

# Plot com heatmap para visualização
plt.figure(figsize=(10, 2))
sns.heatmap(tabela.set_index('Métrica').T, annot=True, cmap='YlOrBr', cbar=False, fmt='.2f')
plt.title("Resumo da Simulação Cinemática - Perfil Agressivo")
plt.yticks(rotation=0)
plt.show()

**A tabela acima apresenta as principais métricas obtidas a partir da simulação com um perfil de velocidade mais agressivo, no qual o veículo acelera intensamente e depois freia. Essa dinâmica permitiu observar o impacto direto da variação de velocidade na trajetória e no desempenho cinemático da bicicleta elétrica.**

**Velocidade Máxima (6.38 m/s):** valor alcançado durante a fase de aceleração intensa, refletindo um comportamento mais esportivo.

**Raio de Curva Estimado (16.29 m):** estimado com base na distância percorrida ao longo da curva, considerando a geometria do movimento.

**Distância Total Percorrida (102.35 m):** resultado da trajetória completa simulada, abrangendo aceleração, curva constante e frenagem.

**Nesta etapa do projeto, vamos implementar um controlador PID (Proporcional–Integral–Derivativo) para simular um sistema de cruise control adaptativo (ACC), responsável por ajustar dinamicamente a força de tração para ajustar a velocidade alvo do veículo. O PID atua sobre o erro entre a velocidade desejada (por exemplo, 15 m/s) e a velocidade real do veículo, gerando uma força de controle que acelera ou desacelera o modelo longitudinal.
Esse mecanismo permite que o veículo:**

**- Acelere de forma suave e inteligente quando está abaixo da meta;**

**- Corrija desvios provocados por inclinações ou variações externas;**

**- Estabilize rapidamente na velocidade desejada, com o mínimo de oscilação.**

**A resposta obtida vai mostrar o sistema atingindo a meta de forma autônoma e eficiente, sendo este um dos pilares para veículos autônomos e elétricos modernos, que demandam controle preciso e adaptativo.**

In [ ]:
# Classe do controlador PID
class PIDController:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp  # Proporcional
        self.Ki = Ki  # Integral
        self.Kd = Kd  # Derivativo
        self.integral = 0 # acumula o erro ao longo do tempo
        self.prev_error = 0 # guarda o erro da iteração anterior, necessário para calcular a derivada

    def control(self, setpoint, current_value, dt):
        error = setpoint - current_value # Calcula o erro entre o valor desejado (ex: velocidade alvo de 15 m/s) e o valor atual (velocidade real da bicicleta).
        self.integral += error * dt # Acumula o erro ao longo do tempo. Ajuda a eliminar o erro residual (offset), especialmente quando o sistema está quase estável
        derivative = (error - self.prev_error) / dt # Mede a taxa de variação do erro. Evita mudanças bruscas e suaviza a resposta do sistema.
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative # Combina o (P, I, D) para calcular o sinal de controle. Esse output representa uma aceleração ou força que o modelo longitudinal vai usar para ajustar a velocidade.
        self.prev_error = error # Atualiza o erro anterior para a próxima iteração.
        return output

# Essa função simula um veículo tentando atingir e manter uma velocidade alvo (ex: 15 m/s) ao longo de um tempo total T, usando o controlador PID implementado
# v_target: velocidade desejada, que o PID tentará manter (default = 15 m/s).
# T: tempo total da simulação (em segundos).
# dt: passo de tempo da simulação (ex: 0.1 s por iteração).

def simulate_pid_control(v_target=15.0, T=20.0, dt=0.1):
    N = int(T / dt) # número de passos na simulação.
    time = np.arange(0, T, dt) # array de tempo para simular de 0 até T com passo dt.

    # Estado do veículo
    v = 0  # velocidade inicial
    v_log = [] # lista para armazenar a velocidade ao longo do tempo.

    # PID: valores ajustáveis conforme o comportamento desejado
    pid = PIDController(Kp=2.0, Ki=0.3, Kd=0.1) # Criando o controlador PID

    for t in time:
        throttle = pid.control(v_target, v, dt) # calcula o sinal de controle com base no erro de velocidade.
        # Simples modelo de resposta do veículo (aceleração proporcional ao throttle)
        a = throttle / 10.0  # aceleração simplificada
        v += a * dt # Velocidade atualizada
        v = max(v, 0)  # não pode ser negativa
        v_log.append(v) # salva a velocidade

    return time, v_log

# Rodar simulação
tempo, velocidades = simulate_pid_control()

In [ ]:
# Gráfico
plt.figure(figsize=(10, 4))
plt.plot(tempo, velocidades, label="Velocidade (m/s)", color='dodgerblue')
plt.axhline(y=15.0, color='red', linestyle='--', label='Velocidade Alvo (15 m/s)')
plt.title("Controle de Velocidade com PID (Cruise Control Simulado)")
plt.xlabel("Tempo (s)")
plt.ylabel("Velocidade (m/s)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**O gráfico acima apresenta a evolução da velocidade do veículo ao longo do tempo em resposta ao controlador PID tentando manter a velocidade alvo de 15 m/s.**

**Linha azul:** velocidade real do veículo (em m/s).

**Linha vermelha tracejada:** referência de velocidade constante desejada (setpoint).

**Aceleração Inicial Forte:** Nos primeiros segundos, o veículo acelera rapidamente em resposta ao erro inicial elevado (termo proporcional agindo com força).

**Ultrapassagem da Meta (Overshoot):** A velocidade atinge cerca de 19 m/s, indicando um overshoot moderado, típico quando o ganho proporcional está mais agressivo ou o ganho integral acumulou erro antes da desaceleração iniciar.

**Estabilização (Tendência):** A velocidade começa a decrescer após o pico, indicando que o controlador integral e derivativo estão agindo para reduzir o erro e evitar oscilação.

**Esse tipo de comportamento é esperado em sistemas com inércia (como um veículo real). O controlador PID está operando bem, pois está controlando suavemente, respondendo rápido ao erro inicial e convergindo para o alvo.**



In [ ]:
# Controlador PID ajustado
class PIDController:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.integral = 0
        self.prev_error = 0

    def control(self, setpoint, current_value, dt):
        error = setpoint - current_value
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
        return output

# Simulação com setpoint variável
def simulate_dynamic_pid_control(T=40.0, dt=0.1):
    time = np.arange(0, T, dt)
    v_target_profile = np.piecewise(
        time,
        [time < 10, (time >= 10) & (time < 20), (time >= 20) & (time < 30), time >= 30],
        [15.0, 5.0, 10.0, 15.0]
    )

    v = 0
    v_log = []
    setpoint_log = []

    # PID mais agressivo e responsivo
    pid = PIDController(Kp=6.0, Ki=0.06, Kd=2.4)

    for i in range(len(time)):
        v_target = v_target_profile[i]
        throttle = pid.control(v_target, v, dt)

        # Limitando aceleração e desaceleração máximas
        max_acc = 6.0   # aceleração máxima m/s²
        max_dec = -9.0  # frenagem m/s²

        acc = np.clip(throttle / 10.0, max_dec, max_acc)
        v += acc * dt
        v = max(v, 0)

        v_log.append(v)
        setpoint_log.append(v_target)

    return time, v_log, setpoint_log

# Rodar a simulação
time, velocidades, setpoints = simulate_dynamic_pid_control()



In [ ]:
# Observando graficamente
plt.figure(figsize=(10, 5))
plt.plot(time, velocidades, label='Velocidade Real (m/s)', linewidth=2)
plt.plot(time, setpoints, '--', label='Velocidade Alvo (m/s)', linewidth=2)
plt.xlabel('Tempo (s)')
plt.ylabel('Velocidade (m/s)')
plt.title('Controle PID Ajustado com Obstáculos / Limites de Velocidade')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**O gráfico acima apresenta o desempenho do controlador PID ajustado para lidar com mudanças dinâmicas de velocidade impostas por obstáculos ou limites de tráfego simulados. O controlador responde com precisão às alterações de setpoint, garantindo uma transição suave e rápida entre diferentes velocidades. As oscilações foram minimizadas com a sintonia adequada dos ganhos, resultando em um sistema robusto e eficiente.**

In [ ]:
# Modelo Longitudinal
class EVLongitudinalModel:
    def __init__(self, mass_kg, wheel_radius_m, CdA, Crr, battery_capacity_Wh,
                 motor_efficiency=0.9, regen_efficiency=0.6):
        self.mass = mass_kg
        self.r = wheel_radius_m
        self.CdA = CdA
        self.Crr = Crr
        self.g = 9.81
        self.rho = 1.225
        self.battery_capacity_Wh = battery_capacity_Wh
        self.motor_eff = motor_efficiency
        self.regen_eff = regen_efficiency
        self.v = 0.0
        self.x = 0.0
        self.battery_Wh = battery_capacity_Wh

    def torque_curve(self, rpm):
        if rpm < 100:
            return 40
        elif rpm < 300:
            return 40 - 0.1 * (rpm - 100)
        else:
            return 20

    def step(self, throttle, brake_force, incline_deg=0.0, dt=0.1):
        rpm = (self.v / (2 * np.pi * self.r)) * 60
        torque = self.torque_curve(rpm)
        F_mot = (torque / self.r) * throttle * self.motor_eff
        theta = np.deg2rad(incline_deg)
        F_drag = 0.5 * self.rho * self.CdA * self.v**2
        F_rr = self.Crr * self.mass * self.g * np.cos(theta)
        F_grade = self.mass * self.g * np.sin(theta)
        F_brake = brake_force
        F_total = F_mot - F_drag - F_rr - F_grade - F_brake
        a = F_total / self.mass
        self.v += a * dt
        self.v = max(self.v, 0)
        self.x += self.v * dt
        power_W = F_mot * self.v
        energy_Wh = (power_W * dt) / 3600
        if throttle > 0:
            self.battery_Wh -= energy_Wh
        elif a < 0 and self.v > 0:
            self.battery_Wh += self.regen_eff * energy_Wh
        self.battery_Wh = np.clip(self.battery_Wh, 0, self.battery_capacity_Wh)
        return self.v

# Modelo Cinemático de Bicicleta
class KinematicBicycleModel:
    def __init__(self, wheelbase_m):
        self.L = wheelbase_m
        self.x = 0.0
        self.y = 0.0
        self.theta = 0.0

    def step(self, v, delta, dt=0.1):
        dx = v * np.cos(self.theta) * dt
        dy = v * np.sin(self.theta) * dt
        dtheta = (v / self.L) * np.tan(delta) * dt
        self.x += dx
        self.y += dy
        self.theta += dtheta
        return self.x, self.y, self.theta

**O controlador Stanley é uma estratégia geométrica amplamente utilizada em sistemas de assistência à condução e veículos autônomos para garantir o seguimento de faixa (Lane Keeping Assist). Ele calcula o ângulo de direção necessário com base no erro lateral (distância perpendicular até o caminho desejado) e no ângulo de orientação do veículo em relação à trajetória.**

In [ ]:
# Controlador Stanley
def stanley_controller(x, y, theta, v, traj_x, traj_y, traj_theta, k=1.5):
    # Distância do veículo a cada ponto da trajetória
    dx = traj_x - x
    dy = traj_y - y
    dists = np.hypot(dx, dy) # calcula a distância euclidiana para cada ponto da trajetória.
    # Seleção do ponto mais próximo na trajetória
    min_idx = np.argmin(dists) # Encontra o índice do ponto mais próximo da trajetória.
    target_x = traj_x[min_idx]
    target_y = traj_y[min_idx]
    target_theta = traj_theta[min_idx] # Extrai a posição e orientação (ângulo) desse ponto: é o ponto-alvo que o veículo deve seguir.
    # Cálculo do erro de orientação (heading error)
    path_yaw = target_theta
    heading_error = path_yaw - theta # heading_error mede a diferença entre a direção do veículo (theta) e a direção da trajetória (path_yaw).
    heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error)) # A normalização com arctan2 garante que o erro fique entre -π e π (evita saltos bruscos).
    # Cálculo do erro lateral (cross-track error)
    dx_front = target_x - x
    dy_front = target_y - y
    cross_track_error = dy_front * np.cos(theta) - dx_front * np.sin(theta) # Esse erro representa o deslocamento lateral do veículo em relação à trajetória, no sistema de coordenadas do próprio veículo.
    # Cálculo do ângulo de direção (delta)
    delta = heading_error + np.arctan2(k * cross_track_error, v + 1e-5) # np.arctan2(k * cross_track_error, v) faz o ajuste de direção. Quanto maior o erro lateral e o ganho k, maior a correção
    return delta                                              # O 1e-5 é um truque para evitar divisão por zero quando v = 0

# Trajetória em S
def generate_s_path(n_points=500): # n_points=500: número de pontos que serão gerados para formar a trajetória (mais pontos = maior resolução).
    x = np.linspace(0, 100, n_points) # Cria um vetor x com n_points igualmente espaçados entre 0 e 100. Representa o avanço ao longo da pista (em metros, por exemplo).
    y = 5 * np.sin(0.1 * x) # Cria um vetor y com um padrão senoidal. Forma uma trajetória em S: suave e ondulada.  amplitude é 5 metros, e a frequência é ajustada com 0.1 para controlar a curvatura.
    theta = np.arctan2(np.gradient(y), np.gradient(x)) # Calcula a orientação da trajetória em cada ponto. np.gradient(y) e np.gradient(x) fornecem uma aproximação da derivada (taxa de variação).
    return x, y, theta                                 # arctan2(dy/dx) retorna o ângulo de inclinação da trajetória em cada ponto — ou seja, a direção que o veículo deveria estar olhando se estivesse seguindo perfeitamente essa curva.

# Simulação
def simulate():
    dt = 0.1 # intervalo de tempo (passo da simulação) em segundos.
    T = 25 # tempo total de simulação (25 segundos).
    N = int(T / dt) # número total de iterações (250 nesse caso).
    traj_x, traj_y, traj_theta = generate_s_path() # Chama a função anterior para gerar a trajetória em S e sua orientação.
    # Inicialização dos modelos
    model_long = EVLongitudinalModel(
        mass_kg=120, wheel_radius_m=0.3, CdA=0.35,
        Crr=0.015, battery_capacity_Wh=500
    )
    model_cine = KinematicBicycleModel(wheelbase_m=1.05)
    # Variáveis para guardar os resutados
    xs, ys, thetas = [], [], []
    vs = []

    for i in range(N): # Laço que roda do tempo t = 0 até t = T em passos de dt.
        # Cálculo da velocidade no modelo longitudinal
        v = model_long.step(throttle=0.6, brake_force=0.0, dt=dt) # Atualiza a velocidade do veículo usando um modelo físico simplificado. Usa 60% do acelerador (throttle=0.6), sem freio (brake_force=0.0).
        # Cálculo do ângulo de direção com Stanley
        delta = stanley_controller(model_cine.x, model_cine.y, model_cine.theta, v,
                                   traj_x, traj_y, traj_theta) # Aplica o controlador Stanley para calcular o ângulo de esterço necessário.
        x, y, theta = model_cine.step(v, delta, dt) # Atualiza a posição e orientação com base na velocidade e ângulo de direção calculados.
        xs.append(x)
        ys.append(y)
        thetas.append(theta)
        vs.append(v)

    # Observando
    plt.figure(figsize=(10, 6))
    plt.plot(traj_x, traj_y, '--', color='black', linewidth=2.5, label="Trajetória Desejada")
    plt.plot(xs, ys, color='orange', linewidth=2.5, label="Trajetória da Bicicleta")
    plt.xlabel("X (m)")
    plt.ylabel("Y (m)")
    plt.title("Controle de Trajetória com Stanley - Trajetória em S")
    plt.legend()
    plt.axis('equal')
    plt.grid(True)
    plt.show()


simulate()

**O gráfico acima representa o desempenho do controlador Stanley aplicado ao modelo cinemático de bicicleta, com velocidades fornecidas dinamicamente pelo modelo longitudinal. A trajetória desejada é uma curva em “S” no plano XY, representada pela linha preta tracejada, enquanto a linha laranja contínua mostra a trajetória efetivamente seguida pelo veículo durante a simulação. Observa-se que o veículo consegue seguir a trajetória com boa precisão ao longo de todo o percurso, mesmo nas curvas mais acentuadas. Esse resultado evidencia a eficiência e estabilidade do controlador, além da integração coerente entre os módulos longitudinal (controle de velocidade) e lateral (controle de direção).**

In [ ]:
# Parâmetros do modelo e controlador
L = 2.5      # entre-eixos do veículo
k = 1.5      # ganho do controlador Stanley
dt = 0.1     # passo de simulação (s). Tempo entre cada iteração da simulação
T = 20       # tempo de simulação
v = 10       # velocidade constante

# Trajetória em S
x_ref = np.linspace(0, 100, 500)
y_ref = 5 * np.sin(0.1 * x_ref)
dy_dx = np.gradient(y_ref, x_ref)
theta_ref = np.arctan2(dy_dx, 1)

# Inicialização do veículo
x, y, theta = 0.0, 0.0, 0.0
x_log, y_log = [x], [y]

# Função do controlador Stanley
def stanley_control(x, y, theta, x_ref, y_ref, theta_ref, k, v):
    dx = x_ref - x
    dy = y_ref - y
    dists = np.hypot(dx, dy)
    idx = np.argmin(dists)

    # Erro de orientação
    path_theta = theta_ref[idx]
    heading_error = path_theta - theta
    heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))

    # Erro lateral com sinal
    dx_front = x_ref[idx] - x
    dy_front = y_ref[idx] - y
    cross_track_error = dy_front * np.cos(theta) - dx_front * np.sin(theta)

    # Controle Stanley
    delta = heading_error + np.arctan2(k * cross_track_error, v + 1e-5)
    return delta, idx

# Simulação do movimento da bicicleta
trajectory = [] # para armazenar as posições (x, y) do veículo a cada instante.
indices = [] # para registrar os índices do ponto da trajetória de referência mais próximo em cada iteração
for _ in np.arange(0, T, dt): # Laço de simulação temporal, que vai de 0 até T com passo dt. Cada passo simula um intervalo de tempo (aqui, 0.1s).
    delta, idx = stanley_control(x, y, theta, x_ref, y_ref, theta_ref, k, v) # Aplica o controlador Stanley:
    # Atualiza a posição da bicicleta com base no modelo cinemático:
    x += v * np.cos(theta) * dt
    y += v * np.sin(theta) * dt
    theta += (v / L) * np.tan(delta) * dt
    x_log.append(x)
    y_log.append(y)
    trajectory.append((x, y))
    indices.append(idx)

# Animação com faixa da pista e marcador visível
fig, ax = plt.subplots(figsize=(10, 6))
lane_width = 3.5

# Trajetória desejada + faixas
ax.plot(x_ref, y_ref, 'k--', linewidth=2.5, label='Trajetória Desejada')
ax.plot(x_ref, y_ref + lane_width, 'gray', linewidth=1.5, linestyle='--')
ax.plot(x_ref, y_ref - lane_width, 'gray', linewidth=1.5, linestyle='--')

# Elementos animados
bike_line, = ax.plot([], [], color='orange', linewidth=2.5, label='Trajetória da Bicicleta')
bike_dot, = ax.plot([], [], 'ro', markersize=8, label='Bicicleta')

# Configurações do gráfico
ax.set_xlim(0, 100)
ax.set_ylim(-20, 20)
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_title("Controle Stanley com Animação e Faixa da Pista")
ax.legend()
ax.grid(True)

# Funções da animação
def init():
    bike_line.set_data([], [])
    bike_dot.set_data([], [])
    return bike_line, bike_dot

def update(frame):
    if frame < len(x_log):
        bike_line.set_data(x_log[:frame], y_log[:frame])
        bike_dot.set_data([x_log[frame]], [y_log[frame]])
    return bike_line, bike_dot

# Criação da animação
ani = FuncAnimation(fig, update, frames=len(x_log), init_func=init, blit=True, interval=50)
HTML(ani.to_jshtml())

**Vamos realizar agora um novo teste com controle de direção realizado com o algoritmo Stanley, vamos ajustar o ângulo de esterçamento com base no erro lateral e de orientação. A velocidade será regulada com um controlador PID dinâmico, que adapta a aceleração conforme o perfil da pista, incluindo uma desaceleração suave para a parada inteligente no final do trajeto.**

In [ ]:
# Parâmetros físicos
L = 1.0       # distância entre eixos (roda dianteira e traseira)
m = 90.0      # massa da bike + piloto (kg)
g = 9.81      # gravidade (m/s²)
rho = 1.225   # densidade do ar (kg/m³)
Cd = 0.9      # coeficiente de arrasto aerodinâmico (quanto menor, menor o arrasto)
A = 0.5       # área frontal (m²) área que enfrenta o vento
Cr = 0.005    # resistência de rolamento dos pneus. Reflete perdas por atrito com o solo
eta = 0.85    # eficiência do motor 85% da energia da bateria é convertida em movimento. O resto é perdido
E_bat = 400000 # energia total da bateria (Joules)
v_vento = 3.0 # velocidade do vento contra (m/s)
raio_roda = 0.7112 # raio da roda (m) essencial para converter torque no eixo do motor em força linear na roda
dt = 0.2 # passo de simulação (s). Tempo entre cada iteração da simulação

# Funções auxiliares essenciais para trabalhar com ângulos e inclinações da pista na simulação.
def normalize_angle(angle): # Normaliza um ângulo para o intervalo.
    return np.arctan2(np.sin(angle), np.cos(angle))

def angle_slope(z, x): # Calcula o ângulo de inclinação da pista (em radianos) com base nos pontos de elevação z e distância horizontal x.
    dz = np.gradient(z) # calcula a variação de altura (derivada aproximada) entre os pontos adjacentes do vetor z.
    dx = np.gradient(x) # calcula a distância horizontal percorrida entre os pontos.
    return np.arctan2(dz, dx) # Calcula a inclinação da pista, evita divisão por zero e calcula o sinal corretamente (subida ou descida).

def pid_speed_control_dynamic(target_v, current_v, integral, prev_error, dt, Kp=0.8, Ki=0.1, Kd=0.3):
    error = target_v - current_v # Calcula o erro instantâneo: quanto falta para a bike atingir a velocidade desejada.
    integral += error * dt # Soma o erro ao longo do tempo. Ajuda a eliminar erro estacionário (quando a velocidade não chega exatamente no alvo)
    derivative = (error - prev_error) / dt # Calcula a velocidade de variação do erro. Ajuda a evitar oscilações.
    acc = Kp * error + Ki * integral + Kd * derivative # Fórmula clássica do PID: combina os três termos para calcular a aceleração desejada.
    return np.clip(acc, -3.0, 2.0), integral, error # Limita a aceleração entre –3.0 m/s² (freada máxima) e 2.0 m/s² (aceleração máxima).

def stanley_control_clipped(x, y, theta, v, cx, cy, current_idx): # calcula o ângulo de esterçamento (delta) que a bike precisa aplicar para seguir a pista, considerando sua posição atual, orientação e velocidade
    dx = cx - x # resulta em um array de distâncias horizontais entre a bike e cada ponto da trajetória.
    dy = cy - y  # resulta um array com as distâncias verticais entre a bike e cada ponto da trajetória
    dists = np.hypot(dx, dy) # Calcula a distância euclidiana entre a bike e cada ponto da trajetória.
    nearest_idx = np.argmin(dists) # retorna o índice do ponto mais próximo da trajetória. A bike vai usar esse ponto como referência para calcular a direção futura (lookahead).

    lookahead_idx = nearest_idx # ponto lookahead é definido como o mais próximo da bike, encontrado na etapa anterior (nearest_idx).
    total_dist = 0.0 # Inicializa a variável total_dist que vai acumular a distância percorrida ao longo dos pontos da trajetória a partir de nearest_idx.
    for i in range(nearest_idx, len(cx) - 1): # acumula distâncias ponto a ponto ao longo da trajetória a partir do ponto atual.
        total_dist += np.hypot(cx[i+1] - cx[i], cy[i+1] - cy[i]) # calcula a distância entre dois pontos consecutivos da trajetória (distância entre i e i+1).
        if total_dist > 4.0: # se total_dist ultrapassa 4 metros,
            lookahead_idx = i # atualiza o lookahead_idx com o índice atual i, indicando o ponto para onde a bike deve "olhar".
            break

    fx, fy = cx[lookahead_idx], cy[lookahead_idx] # são as coordenadas do ponto lookahead, ou seja, o ponto 4 metros à frente que a bike deve seguir.
    path_yaw = np.arctan2(fy - y, fx - x) # Calcula o ângulo da trajetória (path yaw) entre a posição atual da bike (x, y) e o ponto de destino (fx, fy).
    theta_e = normalize_angle(path_yaw - theta) # Calcula o erro de orientação (theta_e), ou seja, a diferença entre path_yaw: direção da trajetória, theta: direção atual da bike.

    front_x = x + L * np.cos(theta) # Essas duas linhas calculam a posição do eixo dianteiro da bike, que é onde o Stanley atua.
    front_y = y + L * np.sin(theta)
    dx_front = fx - front_x # Calcula as distâncias do eixo dianteiro até o ponto lookahead, em x e y.
    dy_front = fy - front_y
    error_front_axle = dx_front * np.sin(path_yaw) - dy_front * np.cos(path_yaw) # Essa é a fórmula clássica do erro lateral da roda dianteira em relação à linha do caminho

    k_dyn = 2.0 / (1.0 + v) # ganho adaptativo que diminui o ganho do erro lateral com a velocidade (mais suave em alta velocidade).
    theta_d = np.arctan2(k_dyn * error_front_axle, v + 1e-5) # ângulo de correção baseado no erro lateral. 1e-5 evita divisão por zero
    delta = normalize_angle(theta_e + theta_d) # o ângulo total de direção necessário para corrigir o curso
    return np.clip(delta, -np.deg2rad(30), np.deg2rad(30)), lookahead_idx # limita o ângulo delta entre -30° e +30° (limite físico do guidão da bike)

# Modelos
class EVLongitudinalModel: # modelo longitudinal do movimento da bike elétrica
    def __init__(self, m, g, rho, Cd, A, Cr, eta, dt, r_roda, v_vento):
        self.m = m
        self.g = g
        self.rho = rho
        self.Cd = Cd
        self.A = A
        self.Cr = Cr
        self.eta = eta
        self.dt = dt
        self.r_roda = r_roda
        self.v_vento = v_vento

    def total_resistance(self, v, slope): # Calcula a força total que resiste ao movimento da bike
        v_rel = v + self.v_vento # velocidade relativa do ar (bike + vento).
        F_drag = 0.5 * self.rho * self.Cd * self.A * v_rel**2 # força de arrasto aerodinâmico
        F_roll = self.Cr * self.m * self.g * np.cos(slope) # Resistência de rolamento
        F_slope = self.m * self.g * np.sin(slope) # Resistência da subida. + quando subindo (resiste), − quando descendo (ajuda)
        return F_drag + F_roll + F_slope # total

    def update(self, v, acc_cmd, slope): # calcula a nova velocidade e força do motor
        F_resist = self.total_resistance(v, slope) # Calcula resistência total
        F_motor = max(5.0, F_resist + self.m * acc_cmd) # Calcula força que o motor precisa aplicar para vencer resistência e gerar aceleração
        a_real = (F_motor - F_resist) / self.m # Aceleração real (considerando a resistência)
        v_new = v + a_real * self.dt # Velocidade nova
        return max(0, v_new), a_real, F_motor # max(0, v_new) garante que não tenha velocidade negativa retorna nova velocidade, aceleração real e força aplicada

class KinematicBicycleModel: # Cuida da trajetória da bike
    def __init__(self, L, dt):
        self.L = L
        self.dt = dt

    def update(self, x, y, theta, v, delta): # atualiza a posição e orientação da bicicleta
        x += v * np.cos(theta) * self.dt # Atualiza a posição horizontal
        y += v * np.sin(theta) * self.dt # Atualiza a posição vertical
        theta += v / self.L * np.tan(delta) * self.dt # Muda a direção da bike com base na velocidade, ângulo do volante delta, e comprimento L.
        return x, y, normalize_angle(theta) # Retorna o novo estado

# Pista
def pista_com_curvas_suaves(step=1.0):
    xt, yt = [], []

    for i in range(30):  # reta inicial
        xt.append(i * step)
        yt.append(0)

    angle = np.radians(15)
    for i in range(30):
        xt.append(xt[-1] + step * np.cos(angle))
        yt.append(yt[-1] + step * np.sin(angle))

    for i in range(20):  # topo
        xt.append(xt[-1] + step)
        yt.append(yt[-1])

    angle = np.radians(-15)
    for i in range(30):
        xt.append(xt[-1] + step * np.cos(angle))
        yt.append(yt[-1] + step * np.sin(angle))

    for i in range(20):  # reta final
        xt.append(xt[-1] + step)
        yt.append(yt[-1])

    return np.array(xt), np.array(yt)

# Simulação
xt, yt = pista_com_curvas_suaves() # retorna as coordenadas da pista final com curvas suaves
zt = yt.copy() # usado para a elevação da pista. Como usamos yt como perfil vertical, copiamos ele
slope_angles = angle_slope(zt, xt) # calcula a inclinação (em radianos) ponto a ponto da pista com angle_slope

# Estados iniciais
x, y = xt[0], yt[0] # posição inicial da bicicleta (primeiro ponto da pista)
theta = np.arctan2(yt[1] - yt[0], xt[1] - xt[0]) # orientação inicial calculada a partir dos dois primeiros pontos da trajetória (derivada direcional)
v = 1.0 # velocidade inicial de 1 m/s
integral = prev_error = 0.0 # inicialização dos termos do PID de velocidade
E_bat_atual = E_bat # energia inicial da bateria (em joules)
x_goal, y_goal = xt[-1], yt[-1] # último ponto da pista, serve como critério de parada da simulação

# Instancia os modelos físicos
ev_model = EVLongitudinalModel(m, g, rho, Cd, A, Cr, eta, dt, raio_roda, v_vento)
bike_model = KinematicBicycleModel(L, dt)

# Logs
x_log, y_log, v_log = [x], [y], [v] # Cria listas para armazenar o histórico da posição x, posição y e velocidade v da bike a cada iteração da simulação
a_log, torque_log, E_bat_log = [], [], [E_bat_atual] # armazena aceleração real da bike (calculada a cada passo), torque aplicado na roda e energia restante da bateria
P_rod_log, P_mot_log, F_motor_log = [], [], [] # Inicializa logs para métricas energéticas e de força. potência útil transmitida à roda, potência demandada do motor e força de tração gerada pelo motor

max_iter = len(xt) * 4 # Define um número máximo de iterações, 4 vezes o número de pontos da pista
dist_to_goal = np.hypot(x_goal - x, y_goal - y) # Calcula a distância Euclidiana entre a posição atual (x, y) e o ponto final da pista (x_goal, y_goal)
iter_count = 0 # Inicializa o contador de iterações

while dist_to_goal > 1.0 and iter_count < max_iter: # Laço que roda até a bike chegar perto do ponto final (dist_to_goal <= 1.0) ou atingir o número máximo de iterações
    current_idx = np.argmin(np.hypot(xt - x, yt - y)) # Acha o índice do ponto da pista mais próximo da posição atual (x, y).
    slope = slope_angles[current_idx] # captura a inclinação (em radianos) do trecho atual da pista (calculada antes com angle_slope).
    slope_deg = np.rad2deg(slope) # Converte a inclinação de radianos para graus, facilitando a lógica de ajuste de velocidade.

    # Calcula o ângulo de direção delta com base na posição e orientação atuais, usando o controlador Stanley com ganho adaptativo.
    delta, _ = stanley_control_clipped(x, y, theta, v, xt, yt, current_idx)
    v_target = 6.0 + np.clip(-slope_deg / 10.0, -2.0, 2.0) # Definição da velocidade-alvo com compensação da inclinação
    v_target = np.clip(v_target, 3.0, 12.0)

    acc_cmd, integral, prev_error = pid_speed_control_dynamic(
        v_target, v, integral, prev_error, dt, Kp=0.6, Ki=0.05, Kd=0.2
    ) # Usa um controlador PID customizado para calcular a aceleração ideal (acc_cmd) para alcançar v_target.

    v_new, a_real, F_motor = ev_model.update(v, acc_cmd, slope) # Atualiza modelo longitudinal da bike
    x, y, theta = bike_model.update(x, y, theta, v_new, delta) # Atualiza posição da bike com o modelo cinemático
    v = v_new # Atualiza a variável v para a nova velocidade, para a próxima iteração do laço

    # Logs
    x_log.append(x)
    y_log.append(y)
    v_log.append(v)
    a_log.append(a_real)
    F_motor_log.append(F_motor)
    torque_log.append(F_motor * raio_roda) # Converte a força do motor (F_motor) em torque (τ = F × r), considerando o raio da roda.
    # Potência na roda e potência do motor
    P_rod = F_motor * v_new
    P_rod_log.append(P_rod)
    P_mot = P_rod / eta if P_rod > 0 else 0
    P_mot_log.append(P_mot)
    E_bat_atual -= P_mot * dt
    E_bat_atual = max(0, E_bat_atual)
    E_bat_log.append(E_bat_atual)

    dist_to_goal = np.hypot(x_goal - x, y_goal - y) # Atualiza a distância até o objetivo final, usada como critério de parada do while
    iter_count += 1 # Incrementa o contador de iteração para evitar laços infinitos





In [ ]:
# Animação
fig, ax = plt.subplots(figsize=(10, 6))
lane_width = 2.0
ax.plot(xt, yt, 'k--', linewidth=2.5, label='Trajeto Desejado')
ax.plot(xt, yt + lane_width, 'gray', linewidth=1.0, linestyle='--')
ax.plot(xt, yt - lane_width, 'gray', linewidth=1.0, linestyle='--')
bike_line, = ax.plot([], [], color='orange', linewidth=2.5, label='Trajetória')
bike_dot, = ax.plot([], [], 'ro', markersize=8, label='Bicicleta')
ax.set_xlim(min(xt)-5, max(xt)+5)
ax.set_ylim(min(yt)-10, max(yt)+10)
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_title("Stanley + PID + Parada Inteligente (Com Modelos Modularizados)")
ax.legend()
ax.grid(True)

def init():
    bike_line.set_data([], [])
    bike_dot.set_data([], [])
    return bike_line, bike_dot

def update(frame):
    if frame < len(x_log):
        bike_line.set_data(x_log[:frame], y_log[:frame])
        bike_dot.set_data([x_log[frame]], [y_log[frame]])
    return bike_line, bike_dot

ani = FuncAnimation(fig, update, frames=range(0, len(x_log), 3), init_func=init, blit=True, interval=50)
HTML(ani.to_jshtml())

**O gráfico apresentado acima mostra o desempenho do sistema de controle da bicicleta elétrica em um trajeto de subida e descida. O objetivo da simulação foi verificar a capacidade do controlador em manter a bicicleta dentro da faixa desejada e realizar a parada com precisão ao final do percurso. A simulação demonstra que a bicicleta foi capaz de seguir o percurso de forma satisfatória, respeitando os limites da pista e realizando a parada de forma segura e controlada.**

**Trajeto Desejado (linha preta tracejada):** Representa a trajetória ideal que a bicicleta deve seguir.

**Trajetória da Bicicleta (linha laranja):** Caminho real percorrido pela bicicleta ao longo da simulação.

**Bicicleta (marcador vermelho):** Posição final da bicicleta ao término do trajeto.

**Faixas de pista (linhas cinza):** Simulam os limites laterais da pista, auxiliando na visualização do desvio lateral.

In [ ]:
# Gráficos
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.plot(v_log, label='Velocidade (m/s)', color='orange')
plt.title('Velocidade ao longo do tempo')
plt.xlabel('Passo')
plt.ylabel('Velocidade')
plt.grid(True)
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(a_log, label='Aceleração (m/s²)', color='green')
plt.title('Aceleração')
plt.xlabel('Passo')
plt.ylabel('Aceleração')
plt.grid(True)
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(torque_log, label='Torque (N·m)', color='blue')
plt.title('Torque aplicado na roda')
plt.xlabel('Passo')
plt.ylabel('Torque')
plt.grid(True)
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(E_bat_log, label='Energia da bateria (J)', color='red')
plt.title('Energia restante da bateria')
plt.xlabel('Passo')
plt.ylabel('Energia')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()



**Velocidade ao longo do tempo:** Notamos a evolução da velocidade da bicicleta em função do tempo de simulação. A curva demonstra o controle eficaz do PID adaptando a velocidade à inclinação da pista, se precebe também que **os** picos e vales indicam trechos de subida (reduz velocidade) e descida (aumenta velocidade).

**Aceleração:** Resposta do sistema de tração ao controle de velocidade, representando as variações instantâneas da velocidade. Valores positivos indicam aceleração (motor tracionando). Valores negativos mostram desaceleração natural ou resistiva (como subida ou resistência do ar). Oscilações suaves sugerem bom ajuste do controlador PID.

**Torque aplicado na roda:** Força de torque entregue à roda motriz ao longo da simulação. Acompanham de perto os trechos onde há maior exigência de força (subidas ou acelerações). Percebemos valores elevados que demonstram esforço do motor para vencer resistências, típico em rampas.

**Energia restante da bateria:** Mostra o consumo cumulativo da bateria durante o trajeto. Queda contínua, mas suave, indica que a bike está operando de forma eficiente. Os pontos de queda mais acentuada coincidem com alta demanda de torque e aceleração.



In [ ]:
# Verificando o menor comprimento entre todas as listas de log
min_len = min(len(x_log), len(v_log), len(a_log), len(torque_log), len(E_bat_log))

# Criando DataFrame
df_sim = pd.DataFrame({
    'x': x_log[:min_len],
    'velocidade': v_log[:min_len],
    'aceleracao': a_log[:min_len],
    'torque': torque_log[:min_len],
    'bateria': E_bat_log[:min_len]
})

# Definindo setores baseados nas posições x da pista
setores_real_corrigido = {
    'Reta Inicial': (0, 30.0),
    'Subida Leve': (30.0, 58.0),
    'Topo': (58.0, 78.0),
    'Descida': (78.0, 108.0),
    'Reta Final': (108.0, 130.0)
}

# Cálculo das métricas por setor
linhas = []
for nome, (x_ini, x_fim) in setores_real_corrigido.items():
    df_setor = df_sim[(df_sim['x'] >= x_ini) & (df_sim['x'] < x_fim)]

    if not df_setor.empty:
        linha = {
            'Trecho': nome,
            'Velocidade Média (m/s)': round(df_setor['velocidade'].mean(), 2),
            'Aceleração Média (m/s²)': round(df_setor['aceleracao'].mean(), 2),
            'Torque Médio (Nm)': round(df_setor['torque'].mean(), 2),
            'Energia Média da Bateria (J)': round(df_setor['bateria'].mean(), 2)
        }
        linhas.append(linha)

# Criando DataFrame final
df_metricas_corretas = pd.DataFrame(linhas)

# Tabela
styled_table = df_metricas_corretas.style.set_caption("📊 Métricas Detalhadas por Setor da Pista")\
    .format({
        'Velocidade Média (m/s)': '{:.2f}',
        'Aceleração Média (m/s²)': '{:.2f}',
        'Torque Médio (Nm)': '{:.2f}',
        'Energia Média da Bateria (J)': '{:,.1f}'
    })\
    .set_table_styles([
        {'selector': 'caption', 'props': [('color', '#0a3c6d'), ('font-size', '16px'), ('font-weight', 'bold')]},
        {'selector': 'th', 'props': [('background-color', '#003366'), ('color', 'white'), ('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '6px'), ('border', '1px solid #ccc')]}
    ])\
    .hide(axis='index')

# Exibir
display(styled_table)


**Definição da nova pista para continuidade do projeto.**

In [ ]:
def pista_oval(
    comprimento_reta=100.0,
    raio_curva=30.0,
    amplitude_s=5.0,
    num_pontos_reta=200,
    num_pontos_curva=100
):

    # Reta superior em formato de 'S'
    x_s1 = np.linspace(0, comprimento_reta, num_pontos_reta)
    y_s1 = amplitude_s * np.sin(2 * np.pi * x_s1 / comprimento_reta)

    # Curva semicircular da direita
    # Curva convexa para a direita.
    angulos_c1 = np.linspace(np.pi / 2, -np.pi / 2, num_pontos_curva)
    # Centro do círculo: (comprimento_reta, -raio_curva)
    x_c1 = comprimento_reta + raio_curva * np.cos(angulos_c1)
    y_c1 = -raio_curva + raio_curva * np.sin(angulos_c1)

    # Reta inferior em formato de 'S'
    x_s2 = np.linspace(comprimento_reta, 0, num_pontos_reta)
    y_s2 = -amplitude_s * np.sin(2 * np.pi * x_s2 / comprimento_reta) - 2 * raio_curva

    # Curva semicircular da esquerda
    # O erro estava aqui. Agora a curva é convexa para a esquerda.
    # Usamos ângulos de pi/2 a 3*pi/2 para desenhar a metade esquerda do círculo.
    angulos_c2 = np.linspace(np.pi / 2, 3 * np.pi / 2, num_pontos_curva)
    # Centro do círculo: (0, -raio_curva)
    x_c2 = 0 + raio_curva * np.cos(angulos_c2)
    y_c2 = -raio_curva + raio_curva * np.sin(angulos_c2)
    # Os pontos são gerados de cima para baixo, então precisamos invertê-los
    # para conectar a parte de baixo (fim da reta S2) com a de cima (início da reta S1).
    x_c2 = x_c2[::-1]
    y_c2 = y_c2[::-1]

    # Concatena todos os segmentos para formar a pista completa
    xt = np.concatenate([x_s1, x_c1, x_s2, x_c2])
    yt = np.concatenate([y_s1, y_c1, y_s2, y_c2])

    return xt, yt

# Gerando a pista
xt, yt = pista_oval(
    comprimento_reta=120.0,
    raio_curva=40.0,
    amplitude_s=8.0
)

# Perfil de elevação
num_total_pontos = len(xt)
zt = np.zeros_like(xt)
metade_pontos = num_total_pontos // 2
zt[:metade_pontos] = 10 * np.sin(np.pi * np.arange(metade_pontos) / metade_pontos)**2


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Gráfico 1: Vista de cima da pista (X-Y)
ax1.plot(xt, yt, 'b-')
ax1.set_title('Vista Superior da Pista Oval com "S" (Corrigida)', fontsize=14)
ax1.set_xlabel('Posição X (m)')
ax1.set_ylabel('Posição Y (m)')
ax1.axis('equal')
ax1.grid(True)

# Gráfico 2: Perfil de Elevação (Z)
ax2.plot(np.arange(num_total_pontos), zt, 'r-')
ax2.set_title('Perfil de Elevação da Pista', fontsize=14)
ax2.set_xlabel('Índice do Ponto na Trajetória')
ax2.set_ylabel('Elevação Z (m)')
ax2.grid(True)

plt.tight_layout()
plt.show()

**Agora vamos implementar uma simulação completa do comportamento dinâmico de uma bicicleta elétrica autônoma percorrendo uma pista oval com variação altimétrica. A pista simulada possui segmentos retos, curvas suaves e elevação senoidal realista. A simulação inicia com velocidade zero, permitindo observar o comportamento natural de partida e aceleração. Durante o percurso, métricas como velocidade, aceleração, torque aplicado e energia restante da bateria são registradas e analisadas a cada iteração. Essa estrutura proporciona uma visão fiel do comportamento de um veículo elétrico leve sob múltiplas condições dinâmicas e de controle. O sistema vai integrar três pilares principais:**

**Modelo Longitudinal de Forças Resistivas:** Considera arrasto aerodinâmico, resistência ao rolamento, inclinação da pista e eficiência do sistema de tração. O modelo calcula a aceleração real com base nas forças motoras e resistivas, além do consumo energético da bateria ao longo do trajeto.

**Modelo Cinemático de Bicicleta:** Simula a trajetória da bicicleta em termos de posição, orientação e deslocamento. Este modelo usa uma abordagem baseada em bicicleta para capturar o comportamento direcional ao longo da pista.

**Controladores PID e Stanley:**

**PID Adaptativo:** Ajusta a velocidade da bicicleta em tempo real, com base na inclinação da pista, priorizando estabilidade, segurança e consumo eficiente.

**Stanley Controller:** Atua na direção, corrigindo o heading (orientação) para garantir o seguimento da trajetória ideal.

In [ ]:
# Parâmetros físicos
L = 1.0
m = 90.0
g = 9.81
rho = 1.225
Cd = 0.9
A = 0.5
Cr = 0.005
eta = 0.85
E_bat = 400000
v_vento = 3.0
raio_roda = 0.7112
dt = 0.2
lookahead_dist = 4.0

# Funções auxiliares
def normalize_angle(angle):
    return np.arctan2(np.sin(angle), np.cos(angle))

def angle_slope(z, x):
    dz = np.gradient(z)
    dx = np.gradient(x)
    return np.arctan2(dz, dx)

def pid_speed_control_dynamic(target_v, current_v, integral, prev_error, dt, Kp=0.6, Ki=0.05, Kd=0.2):
    error = target_v - current_v
    integral += error * dt
    derivative = (error - prev_error) / dt
    acc = Kp * error + Ki * integral + Kd * derivative
    return np.clip(acc, -3.0, 2.0), integral, error

def stanley_control_clipped(x, y, theta, v, cx, cy, current_idx):
    dx = cx - x
    dy = cy - y
    dists = np.hypot(dx, dy)
    nearest_idx = np.argmin(dists)

    lookahead_idx = nearest_idx
    total_dist = 0.0
    N = len(cx)
    for i in range(1, N):
        idx = (nearest_idx + i) % N
        dx = cx[idx] - cx[(nearest_idx + i - 1) % N]
        dy = cy[idx] - cy[(nearest_idx + i - 1) % N]
        total_dist += np.hypot(dx, dy)
        if total_dist > lookahead_dist:
            lookahead_idx = idx
            break

    fx, fy = cx[lookahead_idx], cy[lookahead_idx]
    path_yaw = np.arctan2(fy - y, fx - x)
    theta_e = normalize_angle(path_yaw - theta)

    front_x = x + L * np.cos(theta)
    front_y = y + L * np.sin(theta)
    dx_front = fx - front_x
    dy_front = fy - front_y
    error_front_axle = dx_front * np.sin(path_yaw) - dy_front * np.cos(path_yaw)

    k_dyn = 2.5
    theta_d = np.arctan2(k_dyn * error_front_axle, v + 1e-5)
    delta = normalize_angle(theta_e + theta_d)
    return np.clip(delta, -np.deg2rad(30), np.deg2rad(30)), lookahead_idx

# Modelos
class EVLongitudinalModel:
    def __init__(self, m, g, rho, Cd, A, Cr, eta, dt, r_roda, v_vento):
        self.m, self.g, self.rho, self.Cd = m, g, rho, Cd
        self.A, self.Cr, self.eta, self.dt = A, Cr, eta, dt
        self.r_roda = r_roda
        self.v_vento = v_vento

    def total_resistance(self, v, slope):
        v_rel = v + self.v_vento
        F_drag = 0.5 * self.rho * self.Cd * self.A * v_rel**2
        F_roll = self.Cr * self.m * self.g * np.cos(slope)
        F_slope = self.m * self.g * np.sin(slope)
        return F_drag + F_roll + F_slope

    def update(self, v, acc_cmd, slope):
        F_resist = self.total_resistance(v, slope)
        F_motor = max(5.0, F_resist + self.m * acc_cmd)
        a_real = (F_motor - F_resist) / self.m
        v_new = v + a_real * self.dt
        return max(0, v_new), a_real, F_motor

class KinematicBicycleModel:
    def __init__(self, L, dt):
        self.L, self.dt = L, dt

    def update(self, x, y, theta, v, delta):
        x += v * np.cos(theta) * self.dt
        y += v * np.sin(theta) * self.dt
        theta += v / self.L * np.tan(delta) * self.dt
        return x, y, normalize_angle(theta)

# Pista oval
def pista_oval(comprimento_reta=120.0, raio_curva=40.0, amplitude_s=8.0, num_pontos_reta=200, num_pontos_curva=100):
    x_s1 = np.linspace(0, comprimento_reta, num_pontos_reta)
    y_s1 = amplitude_s * np.sin(2 * np.pi * x_s1 / comprimento_reta)

    angulos_c1 = np.linspace(np.pi / 2, -np.pi / 2, num_pontos_curva)
    x_c1 = comprimento_reta + raio_curva * np.cos(angulos_c1)
    y_c1 = -raio_curva + raio_curva * np.sin(angulos_c1)

    x_s2 = np.linspace(comprimento_reta, 0, num_pontos_reta)
    y_s2 = -amplitude_s * np.sin(2 * np.pi * x_s2 / comprimento_reta) - 2 * raio_curva

    angulos_c2 = np.linspace(np.pi / 2, 3 * np.pi / 2, num_pontos_curva)
    x_c2 = raio_curva * np.cos(angulos_c2)[::-1]
    y_c2 = -raio_curva + raio_curva * np.sin(angulos_c2)[::-1]

    xt = np.concatenate([x_s1, x_c1, x_s2, x_c2])
    yt = np.concatenate([y_s1, y_c1, y_s2, y_c2])
    return xt, yt

# Simulação
xt, yt = pista_oval()
zt = np.zeros_like(xt)
metade = len(xt) // 2
zt[:metade] = 10 * np.sin(np.pi * np.arange(metade) / metade)**2
slope_angles = angle_slope(zt, xt)

x, y = xt[0], yt[0]
theta = np.arctan2(yt[1] - yt[0], xt[1] - xt[0])
v = 0.0
integral = prev_error = 0.0
E_bat_atual = E_bat

x_log, y_log, v_log = [x], [y], [v]
a_log, torque_log, E_bat_log = [], [], [E_bat_atual]
P_rod_log, P_mot_log, F_motor_log = [], [], []

model_ev = EVLongitudinalModel(m, g, rho, Cd, A, Cr, eta, dt, raio_roda, v_vento)
model_bike = KinematicBicycleModel(L, dt)

max_iter = 5000
iter_count = 0
dist_to_start = 1e9

while iter_count < max_iter:
    current_idx = np.argmin(np.hypot(xt - x, yt - y))
    slope = slope_angles[current_idx]
    slope_deg = np.rad2deg(slope)

    delta, _ = stanley_control_clipped(x, y, theta, v, xt, yt, current_idx)
    v_target = np.clip(6.0 - slope_deg / 10.0, 3.0, 12.0)
    acc_cmd, integral, prev_error = pid_speed_control_dynamic(v_target, v, integral, prev_error, dt)

    v_new, a_real, F_motor = model_ev.update(v, acc_cmd, slope)
    x, y, theta = model_bike.update(x, y, theta, v_new, delta)
    v = v_new

    x_log.append(x); y_log.append(y); v_log.append(v); a_log.append(a_real)
    F_motor_log.append(F_motor); torque_log.append(F_motor * raio_roda)
    P_rod = F_motor * v_new
    P_mot = P_rod / eta if P_rod > 0 else 0
    E_bat_atual -= P_mot * dt; E_bat_atual = max(0, E_bat_atual)

    P_rod_log.append(P_rod); P_mot_log.append(P_mot); E_bat_log.append(E_bat_atual)
    iter_count += 1

    # Verifica se a bike está de volta ao ponto inicial
    dist_to_start = np.hypot(x - xt[0], y - yt[0])
    if iter_count > 300 and dist_to_start < 5.0:
        print(f"Simulação encerrada após {iter_count} iterações. Volta completa.")
        break




In [ ]:
# Simulação de dados reais
steps = len(x_log)
df_empty = pd.DataFrame(columns=["Passo", "Velocidade (m/s)", "Aceleração (m/s²)", "Torque (Nm)", "Energia (J)"])

# Criando a figura e subplots
fig, (ax_path, ax_table) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [3, 2]})
lane_width = 2.0

# Trajeto da pista
ax_path.plot(xt, yt, 'k--', linewidth=2, label='Trajeto Desejado')
ax_path.plot(xt, yt + lane_width, 'gray', linestyle='--', linewidth=1)
ax_path.plot(xt, yt - lane_width, 'gray', linestyle='--', linewidth=1)

bike_line, = ax_path.plot([], [], color='orange', linewidth=2.5, label='Trajetória da Bike')
bike_dot, = ax_path.plot([], [], 'ro', markersize=8)
ax_path.set_xlim(min(xt) - 5, max(xt) + 5)
ax_path.set_ylim(min(yt) - 15, max(yt) + 15)
ax_path.set_title("Simulação: Stanley + PID (Bike na Pista Oval)")
ax_path.set_xlabel("X (m)")
ax_path.set_ylabel("Y (m)")
ax_path.grid(True)
ax_path.legend()

# Tabela textual animada
table_text = ax_table.text(0.01, 0.95, '', va='top', ha='left', fontsize=10, fontfamily='monospace')
ax_table.axis('off')

# Função de atualização
def update(frame):
    if frame >= steps:
        return bike_line, bike_dot, table_text

    # Atualiza a posição
    bike_line.set_data(x_log[:frame], y_log[:frame])
    bike_dot.set_data([x_log[frame]], [y_log[frame]])

    # Atualiza a tabela
    dados = {
        "Passo": frame,
        "Velocidade (m/s)": round(v_log[frame], 2),
        "Aceleração (m/s²)": round(a_log[frame], 2),
        "Torque (Nm)": round(torque_log[frame], 2),
        "Energia (J)": round(E_bat_log[frame], 1)
    }
    df_display = pd.concat([df_empty, pd.DataFrame([dados])], ignore_index=True).tail(10)
    table_text.set_text(df_display.to_string(index=False))

    return bike_line, bike_dot, table_text

# Criando animação
ani = FuncAnimation(fig, update, frames=range(0, steps, 2), blit=False, interval=100)
HTML(ani.to_jshtml())

In [ ]:
# Observando graficamente a simulação
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.plot(v_log, label='Velocidade (m/s)', color='orange')
plt.title('Velocidade ao Longo do Tempo')
plt.xlabel('Passo de Simulação')
plt.ylabel('Velocidade (m/s)')
plt.grid(True)
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(a_log, label='Aceleração (m/s²)', color='green')
plt.title('Aceleração Real')
plt.xlabel('Passo de Simulação')
plt.ylabel('Aceleração (m/s²)')
plt.grid(True)
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(torque_log, label='Torque na Roda (N·m)', color='blue')
plt.title('Torque Aplicado na Roda')
plt.xlabel('Passo de Simulação')
plt.ylabel('Torque (N·m)')
plt.grid(True)
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(E_bat_log, label='Energia da Bateria (J)', color='red')
plt.title('Energia Restante da Bateria')
plt.xlabel('Passo de Simulação')
plt.ylabel('Energia (J)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.suptitle("Desempenho da Bike Elétrica na Pista Oval com Curvas em S", fontsize=14, y=1.02)
plt.show()


**Velocidade ao longo do tempo:** Percebemos que a curva reflete a evolução da velocidade da bicicleta elétrica ao longo da simulação, temos um controle eficaz do sistema PID, que ajusta dinamicamente a velocidade conforme a inclinação do terreno. Os picos estão associados a trechos de descida, enquanto as quedas indicam subidas ou curvas mais exigentes. A transição suave entre essas fases demonstra boa calibração do controlador.

**Aceleração:** Esse gráfico representa a resposta do sistema de tração frente ao controle de velocidade. Acelerações positivas indicam momentos em que o motor está impulsionando a bike (especialmente em subidas), enquanto desacelerações ocorrem por ação da resistência do ar, gravidade ou estratégia de controle em curvas. As oscilações suaves e coerentes evidenciam um controle PID bem ajustado.

**Torque aplicado na roda:** O torque aplicado à roda motriz acompanha a demanda de força ao longo do trajeto, pois os picos de torque surgem nos momentos em que o motor precisa vencer subidas ou recuperar velocidade após curvas. Já os vales refletem trechos mais planos ou descendentes, onde o esforço do motor é menor. Percebemos que a curva mostra claramente os pontos críticos de esforço mecânico.

**Energia restante da bateria:** A curva de energia mostra o consumo acumulado ao longo da pista. A queda gradual indica operação eficiente, sem desperdício excessivo. Notam-se trechos de maior declínio, que coincidem com momentos de maior torque e aceleração que são típicos de subidas e retomadas. A estratégia de controle contribui para o uso inteligente da energia disponível.

In [ ]:
# Criando DataFrame com os logs completos (até a iteração 301)
min_len = min(len(x_log), len(v_log), len(a_log), len(torque_log), len(E_bat_log))

df_trajeto = pd.DataFrame({
    'Passo': np.arange(min_len),
    'Velocidade (m/s)': v_log[:min_len],
    'Aceleração (m/s²)': a_log[:min_len],
    'Torque (Nm)': torque_log[:min_len],
    'Energia da Bateria (J)': E_bat_log[:min_len]
})

# Selecionando 10 amostras distribuídas ao longo do tempo
df_amostras = df_trajeto.iloc[np.linspace(0, min_len-1, 10, dtype=int)].reset_index(drop=True)

# Exibindo em tabela
display(df_amostras.style.set_caption("📊 Evolução dos Parâmetros ao Longo da Simulação")
        .format({
            'Velocidade (m/s)': '{:.2f}',
            'Aceleração (m/s²)': '{:.2f}',
            'Torque (Nm)': '{:.2f}',
            'Energia da Bateria (J)': '{:,.1f}'
        })
        .set_table_styles([
            {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
            {'selector': 'th', 'props': [('background-color', '#003366'), ('color', 'white'), ('padding', '8px')]},
            {'selector': 'td', 'props': [('padding', '6px'), ('border', '1px solid #ccc')]}
        ])
        .hide(axis='index'))
